In [10]:
import sys
sys.path.append('../')

from model import *
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import pickle as pkl
import matplotlib.pyplot as plt
import sklearn


In [11]:
#parameters
pi_1 = 4.0
pi_2= 0.8
tau = 1.0
rho= 0.02
K=10
params = {'layers': []}

In [12]:
with open('../../../data/ChildWelfare/X_preprocess.pkl', 'rb') as handle:
    X,screener_ids,refer_ids,Y_observed,Y_human,Y_serv,Y_sub,colnames = pkl.load(handle)


In [13]:
def partition_data(X,  refer_ids, screener_ids, Y_human, Y_observed):
    refer_ids_train, refer_ids_test = sklearn.model_selection.train_test_split(np.unique(refer_ids),test_size = 0.25)

    X_train = X[[(r in refer_ids_train) for r in refer_ids],:]
    X_test = X[[(r in refer_ids_test) for r in refer_ids],:]

    Y_h_train = np.array(Y_human[[(r in refer_ids_train) for r in refer_ids]])
    Y_h_test = np.array(Y_human[[(r in refer_ids_test) for r in refer_ids]])

    Y_obs_train = np.array(Y_observed[[(r in refer_ids_train) for r in refer_ids]])
    Y_obs_test = np.array(Y_observed[[(r in refer_ids_test) for r in refer_ids]])

    screener_ids_train = np.array(np.array(screener_ids)[[(r in refer_ids_train) for r in refer_ids]])
    screener_ids_test = np.array(np.array(screener_ids)[[(r in refer_ids_train) for r in refer_ids]])

    Y_h_train = np.transpose(Y_h_train)[0]
    Y_h_test = np.transpose(Y_h_test)[0]
    Y_obs_train = np.transpose(Y_obs_train)[0]
    Y_obs_test = np.transpose(Y_obs_test)[0]
    
    return(refer_ids_train, refer_ids_test,X_train,X_test,Y_h_train,Y_h_test,Y_obs_train,Y_obs_test,screener_ids_train,screener_ids_test)

In [14]:
refer_ids_train, refer_ids_test,X_train,X_test,Y_h_train,Y_h_test,Y_obs_train,Y_obs_test, screener_ids_train,screener_ids_test = partition_data(X,  refer_ids, screener_ids, Y_human, Y_observed)
    

In [15]:
for l1_penalty in [0.001, 0.01]:
    try:
        model = BinaryMLP(**params)
        model = model.fit(X_train, tar_train['D'], screener_ids_train, l1_penalty = l1_penalty)
        break
    except Exception as e:
        print(e, l1_penalty)
        pass

Loss: 0.598: 100%|██████████| 1/1 [00:00<00:00,  3.04it/s]


In [ ]:
def amalgamate_and_hybrid(X,  refer_ids, screener_ids, Y_human, Y_observed, fit_classifier ='logit', f_y_hyb = 'retrain'):
    #partition data
    
    if fit_classifier =='logit':
        clf_obs = linear_model.LogisticRegression(penalty = 'l2', solver = 'liblinear')
        clf_amalg = linear_model.LogisticRegression(penalty = 'l2', solver = 'liblinear')
        clf_human = linear_model.LogisticRegression(penalty = 'l2', solver = 'liblinear')
    if fit_classifier =='rf':
        param_grid = {'min_samples_leaf':[10, 25, 50]}
        clf_obs = RandomForestClassifier(random_state=0)
        clf_amalg = RandomForestClassifier(random_state=0)
        clf_human = RandomForestClassifier(random_state=0)
        
    #calculate consistency through cv . in train fols
    M_inf, Y_pred_h, Agr1, Agr2 = label_agreeability(X_train,Y_h_train,screener_ids_train,0.05)
    high_agr = np.where((Agr2[0]> pi_2) & (Agr1[0]> pi_1))
    
    #amalgamate labels
    Y_amalg_train = np.copy(Y_obs_train)
    high_agr_correct = [idx for idx in high_agr[0] if np.abs(Y_pred_h[0][idx]-Y_h_train[idx]) < 0.05 ]
    Y_amalg_train[high_agr_correct] = (1-tau)*Y_obs_train[high_agr_correct]+tau*Y_h_train[high_agr_correct]
    
    #train model on observed labels (on portion that is screened-in)
    if fit_classifier =='logit':
        clf_obs.fit(X_train[Y_h_train==1,:], Y_obs_train[Y_h_train==1])
    if fit_classifier =='rf':
        grid_search = GridSearchCV(estimator = clf_obs, param_grid = param_grid, 
                          cv = 3, n_jobs = 10, verbose = 1)
        grid_search.fit(X_train[Y_h_train==1,:], Y_obs_train[Y_h_train==1])
        clf_obs = grid_search.best_estimator_
    
    Y_pred_obs_train = clf_obs.predict_proba(X_train[Y_h_train==1,:])[:, 1]
    Y_pred_obs_test = clf_obs.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = metrics.roc_curve(Y_obs_train[Y_h_train==1], Y_pred_obs_train)
    print("AUC_train f_y", metrics.auc(fpr, tpr))
    
    #train model on amalgamated labels
    idx_amalg_train = np.unique(np.concatenate((np.where(Y_h_train==1)[0], np.array(high_agr_correct))))
    idx_amalg_train = idx_amalg_train.astype(int)
    if fit_classifier =='logit':
        clf_amalg.fit(X_train[idx_amalg_train,:], Y_amalg_train[idx_amalg_train])
    if fit_classifier =='rf':
        grid_search = GridSearchCV(estimator = clf_amalg, param_grid = param_grid, 
                          cv = 3, n_jobs = 10, verbose = 1)
        grid_search.fit(X_train[idx_amalg_train,:], Y_amalg_train[idx_amalg_train])
        clf_amalg = grid_search.best_estimator_
    
    Y_pred_amalg_train = clf_amalg.predict_proba(X_train[idx_amalg_train,:])[:, 1]
    Y_pred_amalg_test= clf_amalg.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = metrics.roc_curve(Y_amalg_train[idx_amalg_train], Y_pred_amalg_train)
    print("AUC_train f_A", metrics.auc(fpr, tpr))
    
    #train model on human decisions
    if fit_classifier =='logit':
        clf_human.fit(X_train, Y_h_train)
    if fit_classifier =='rf':
        grid_search = GridSearchCV(estimator = clf_human, param_grid = param_grid, 
                          cv = 3, n_jobs = 10, verbose = 1)
        grid_search.fit(X_train, Y_h_train)
        clf_human = grid_search.best_estimator_
    
    Y_pred_h_train = clf_human.predict_proba(X_train)[:, 1]
    Y_pred_h_test = clf_human.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = metrics.roc_curve(Y_h_train, Y_pred_h_train)
    print("AUC_train f_h", metrics.auc(fpr, tpr))
    
    #Determine which test instances are in set A
    M_inf_test, Y_pred_h_test_L1, Agr1_test, Agr2_test = label_agreeability_test(X_train, X_test, Y_h_train, screener_ids_train, rho)
    #idx of high_agreement instances
    high_agr_test = np.where((Agr2_test[0]> pi_2) & (Agr1_test[0]> pi_1))[0]
    
    
    #hybrid predictions: if x in A->f_h, if x not in A->f_y
    Y_pred_hybrid_test = np.copy(Y_pred_obs_test)
    
    
    #Assign predictions of human model to high-consistency set for hybrid approach 
    #uses same model as the one used to obtain set A_test
    if f_y_hyb=='retrain':
        Y_pred_hybrid_test[high_agr_test] = Y_pred_h_test[high_agr_test]
    else:
        Y_pred_hybrid_test[high_agr_test] = Y_pred_h_test_L1[high_agr_test]
    #uses separate human model (L2) to ensure comparability with amalgamated model
    
    
    
    return(Y_pred_hybrid_test, Y_pred_obs_test, Y_pred_amalg_test, Y_pred_h_test, refer_ids_train, refer_ids_test)